In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab_Notebooks/Author_style/author_style

/content/drive/MyDrive/Colab_Notebooks/Author_style/author_style


In [ ]:
!pip install unidecode

     |████████████████████████████████| 241 kB 7.2 MB/s 


In [ ]:
from author_style.utils import *
from author_style.preprocessing import *


In [4]:
import joblib 

from google.colab import files
uploaded = files.upload()

Saving X.pkl to X (3).pkl


In [3]:
import io 
y=io.BytesIO(uploaded['y.pkl'])

In [5]:
import io 
X=io.BytesIO(uploaded['X.pkl'])

In [6]:
y = joblib.load(y)
X = joblib.load(X)

### DATA

In [ ]:
X

array([["pendant que nous franchissions la porte du nord, dediee a ishtar, deesse de la fecondite, la vierge, toujours l'esprit en eveil, me donna une idee qui recut immediatement force de loi. pour compenser la fatigue que pourrait causer le jacassement universel, la derniere semaine de chaque mois serait vouee entierement au silence. un silence de reve suspendrait, sur la terre, le grondement des mers, le fracas des cascades, le souffle des vents, le vrombissement des avions, le bruit des trains, des voitures, des machines. il eteindrait, dans l'univers, la musique des spheres et tout fremissement de vie. cette semaine mensuelle de silence serait vouee a l'amour. en retenant leur souffle, les amoureux redoubleraient de regards, de caresses. ils ne seraient plus que rayonnements d'ame, scintillements d'indicible. quand ils retourneraient au monde de la parole, ils garderaient une telle nostalgie du silence qu'ils croiraient en dire infiniment moins avec leurs mots qu'auparavant avec l

In [7]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import wordnet

#-------------------------------------------------------

try:
  nltk.data.find('punkt')
  nltk.data.find('stopwords')
  nltk.data.find('averaged_perceptron_tagger')
  nltk.data.find('wordnet')
except LookupError:
  nltk.download('punkt', quiet=True)
  nltk.download('stopwords', quiet=True)
  nltk.download('averaged_perceptron_tagger', quiet=True)
  nltk.download('wordnet', quiet=True)

X_token = [word_tokenize(list(elem)[0]) for elem in X]
X_token_len = [ len(sequence) for sequence in X_token]
print("X_token_len =",len(X_token_len))

X_token_len = 17779


In [ ]:
print(y.shape)
print(X.shape)

(17779,)
(17779, 1)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_token,y, test_size=0.3)

In [ ]:
X_train[0]

In [9]:
from gensim.models import Word2Vec
word2vec= Word2Vec(X_train, min_count=6, size=40)

In [10]:
import numpy as np
def embed_sentence(word2vec, sentence):
    # $CHALLENGIFY_BEGIN
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

In [11]:
def embedding(word2vec, sentences):
    # $CHALLENGIFY_BEGIN
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

y_train_encoded=to_categorical(y_train)
y_test_encoded=to_categorical(y_test)

X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', maxlen=1000)
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', maxlen=1000)

### Baseline Model

In [ ]:
import numpy as np

np.unique(y_train, return_counts=True)

In [ ]:
baseline_accuracy=1/25
baseline_accuracy

0.04

## RNN

### MODEL LSTM

In [ ]:
X_train_pad.shape

(12445, 1000, 50)

In [ ]:
len(X_train_pad[0])

1000

In [ ]:
X_train_pad.shape

(12445, 1000, 50)

In [ ]:
y_train_encoded.shape

(12445, 25)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Masking, AveragePooling1D, TimeDistributed, Dropout
from tensorflow.keras.optimizers import RMSprop

model=Sequential([
                  (Masking(mask_value=0.0)),
                  (LSTM(units=100, activation='tanh', return_sequences=True)),
                  (LSTM(200, activation='tanh', return_sequences=True)),
                  (LSTM(300, activation='tanh')),
                  #(AveragePooling1D()),
                  (Dropout(.2)),
                  #(Dense(300, activation='relu')),
                  (Dense(100, activation='relu')),
                  (Dense(50, activation='relu')),
                  (Dense(25, activation='softmax'))
])
optimizer=RMSprop(0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics='accuracy')

#model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

es=EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy')

model.fit(X_train_pad, y_train_encoded, epochs=30, batch_size=16, callbacks=[es], validation_split=0.2)

In [ ]:
model.evaluate(X_test_pad, y_test_encoded)

167/167 [==============================] - 6s 36ms/step - loss: 2.0377 - accuracy: 0.4456


[2.0376901626586914, 0.44563180208206177]

BIDIRECTIONAL LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Masking, AveragePooling1D, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import RMSprop

model_1=Sequential([
                  (Masking(mask_value=0.0)),
                  (Bidirectional(LSTM(units=100, activation='tanh', return_sequences=True))),
                  (Bidirectional(LSTM(units=200, activation='tanh', return_sequences=True))),
                  (Bidirectional(LSTM(units=300, activation='tanh'))),
                  (Dropout(.2)),
                  (Dense(100, activation='relu')),
                  (Dropout(.2)),
                  (Dense(50, activation='relu')),
                  (Dense(25, activation='softmax'))
])
optimizer=RMSprop(0.0001)

model_1.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics='accuracy')

In [ ]:
from keras.callbacks import EarlyStopping

es=EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy')

model_1.fit(X_train_pad, y_train_encoded, epochs=20, batch_size=16, callbacks=[es], validation_split=0.2)

Epoch 1/20
623/623 [==============================] - 123s 162ms/step - loss: 3.0052 - accuracy: 0.1229 - val_loss: 2.7974 - val_accuracy: 0.1784
Epoch 2/20
623/623 [==============================] - 95s 153ms/step - loss: 2.7022 - accuracy: 0.2143 - val_loss: 2.6403 - val_accuracy: 0.2218
Epoch 3/20
623/623 [==============================] - 96s 154ms/step - loss: 2.4666 - accuracy: 0.2721 - val_loss: 2.3810 - val_accuracy: 0.2816
Epoch 4/20
623/623 [==============================] - 95s 153ms/step - loss: 2.3042 - accuracy: 0.3101 - val_loss: 2.2586 - val_accuracy: 0.3126
Epoch 5/20
623/623 [==============================] - 95s 153ms/step - loss: 2.1540 - accuracy: 0.3535 - val_loss: 2.2276 - val_accuracy: 0.3335
Epoch 6/20
623/623 [==============================] - 96s 154ms/step - loss: 2.0456 - accuracy: 0.3863 - val_loss: 2.1423 - val_accuracy: 0.3640
Epoch 7/20
623/623 [==============================] - 95s 153ms/step - loss: 1.9397 - accuracy: 0.4165 - val_loss: 2.0643 - val_a

In [ ]:
model_1.evaluate(X_test_pad, y_test_encoded)

167/167 [==============================] - 13s 76ms/step - loss: 2.3090 - accuracy: 0.4426


[2.308995008468628, 0.44263216853141785]

Boucle FOR

In [ ]:
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import LSTM, Dense, Masking, AveragePooling1D, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

results={}
sizes=[25, 50, 75, 100, 150, 200]
for size in sizes:
  word2vec= Word2Vec(X_train, min_count=1, size=size)

  X_train_embeded = embedding(word2vec, X_train)
  X_test_embeded = embedding(word2vec, X_test)

  X_train_pad = pad_sequences(X_train_embeded, dtype='float32', padding='post', maxlen=1000)
  X_test_pad = pad_sequences(X_test_embeded, dtype='float32', padding='post', maxlen=1000)

  model=Sequential([
                  (Masking(mask_value=0.0)),
                  (LSTM(units=50, activation='tanh', return_sequences=True)),
                  (LSTM(75, activation='tanh', return_sequences=True)),
                  (LSTM(100, activation='tanh', return_sequences=True)),
                  #(AveragePooling1D()),
                  (Dense(100, activation='relu')),
                  (Dense(50, activation='relu')),
                  (Dense(25, activation='softmax'))
])

  
  
  
  model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics='accuracy')
  
  es=EarlyStopping(patience=8, restore_best_weights=True, monitor='val_accuracy')

  model.fit(X_train_pad, y_train_encoded, epochs=10, batch_size=16, callbacks=[es], validation_split=0.2)

  y_pred=model.predict(X_test_pad)
  accuracy=accuracy_score(X_test_pad, y_test_encoded)

  results[size]=accuracy


## CNN

MODEL CONV1D

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Masking, Conv1D, Flatten, Dropout, MaxPool1D
from tensorflow.keras.optimizers import Adam

model_2=Sequential([
                  (Masking(mask_value=0.0)),
                  (Conv1D(32, kernel_size=3)),
                  (Conv1D(32, kernel_size=3)),
                  (Dropout(.2)),
                  (MaxPool1D(20)),
                  (Flatten()),
                  #(Dropout(.2)),

                  (Dense(50, activation='relu')),
                  (Dense(25, activation='softmax'))])

optimizer=Adam(0.001)

model_2.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics='accuracy')



In [16]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

es=EarlyStopping(patience=15, restore_best_weights=True, monitor='val_accuracy')
rlp=ReduceLROnPlateau(patience=2, factor=0.3, monitor='loss', verbose=True)
model_2.fit(X_train_pad, y_train_encoded, epochs=50, batch_size=64, callbacks=[es, rlp], validation_split=0.2)

Epoch 1/50
156/156 [==============================] - 2s 11ms/step - loss: 2.9465 - accuracy: 0.1322 - val_loss: 2.7876 - val_accuracy: 0.1543
Epoch 2/50
156/156 [==============================] - 1s 8ms/step - loss: 2.6797 - accuracy: 0.1979 - val_loss: 2.6313 - val_accuracy: 0.2113
Epoch 3/50
156/156 [==============================] - 1s 8ms/step - loss: 2.4901 - accuracy: 0.2501 - val_loss: 2.4750 - val_accuracy: 0.2571
Epoch 4/50
156/156 [==============================] - 1s 8ms/step - loss: 2.3251 - accuracy: 0.2966 - val_loss: 2.4004 - val_accuracy: 0.2720
Epoch 5/50
156/156 [==============================] - 1s 8ms/step - loss: 2.1984 - accuracy: 0.3315 - val_loss: 2.3358 - val_accuracy: 0.2857
Epoch 6/50
156/156 [==============================] - 1s 8ms/step - loss: 2.0921 - accuracy: 0.3633 - val_loss: 2.2613 - val_accuracy: 0.3021
Epoch 7/50
156/156 [==============================] - 1s 8ms/step - loss: 2.0095 - accuracy: 0.3844 - val_loss: 2.3340 - val_accuracy: 0.3061
Epoch

In [17]:
model_2.evaluate(X_test_pad, y_test_encoded)

167/167 [==============================] - 1s 4ms/step - loss: 2.4350 - accuracy: 0.3127


[2.435047149658203, 0.3127109110355377]